The goal of the project is to analyze the results of A/B testing using statistical methods in Python and to create a visualization that demonstrates the key conversion metrics.

##Preparing the data

###Importing libraries

In [ ]:
!pip install --upgrade google-cloud-bigquery
from google.colab import auth
from google.cloud import bigquery
import numpy as np
import pandas as pd
import numpy as np
import statsmodels.api as sm

###Executing an SQL query on the database

In [ ]:
auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")

In [ ]:
query = """
with session_info as (
 Select
     s.date,
     s.ga_session_id,
     sp.country,
     sp.device,
     sp.continent,
     sp.channel,
     ab.test,
     ab.test_group
 from `DA.ab_test`  ab
 join `DA.session`  s
   on ab.ga_session_id = s.ga_session_id
 join `DA.session_params`  sp
   on sp.ga_session_id = ab.ga_session_id
),
session_with_orders as (
 SELECT
     session_info.date,
     session_info.country,
     session_info.device,
     session_info.continent,
     session_info.channel,
     session_info.test,
     session_info.test_group,
     count(distinct o.ga_session_id) as session_with_orders
 from `DA.order` o
 join session_info
   on o.ga_session_id = session_info.ga_session_id
 group by
     session_info.date,
     session_info.country,
     session_info.device,
     session_info.continent,
     session_info.channel,
     session_info.test,
     session_info.test_group
),
events as (
 Select
     session_info.date,
     session_info.country,
     session_info.device,
     session_info.continent,
     session_info.channel,
     session_info.test,
     session_info.test_group,
     sp.event_name,
     count(sp.ga_session_id) as event_cnt
 from `DA.event_params`  sp
 join session_info
   on sp.ga_session_id = session_info.ga_session_id
 group by
     session_info.date,
     session_info.country,
     session_info.device,
     session_info.continent,
     session_info.channel,
     session_info.test,
     session_info.test_group,
     sp.event_name
),
session as (
 Select
     session_info.date,
     session_info.country,
     session_info.device,
     session_info.continent,
     session_info.channel,
     session_info.test,
     session_info.test_group,
     count(distinct session_info.ga_session_id) as session_cnt
 from session_info
 group by
     session_info.date,
     session_info.country,
     session_info.device,
     session_info.continent,
     session_info.channel,
     session_info.test,
     session_info.test_group
),
account as (
Select
     session_info.date,
     session_info.country,
     session_info.device,
     session_info.continent,
     session_info.channel,
     session_info.test,
     session_info.test_group,
     count(distinct acs.ga_session_id) as new_account_cnt
 from `DA.account_session`  acs
 join session_info
   on acs.ga_session_id = session_info.ga_session_id
 group by
     session_info.date,
     session_info.country,
     session_info.device,
     session_info.continent,
     session_info.channel,
     session_info.test,
     session_info.test_group)

Select
   session_with_orders.date,
   session_with_orders.country,
   session_with_orders.device,
   session_with_orders.continent,
   session_with_orders.channel,
   session_with_orders.test,
   session_with_orders.test_group,
   'session with orders' as event_name,
   session_with_orders.session_with_orders as value
from session_with_orders
union all
Select
   events.date,
   events.country,
   events.device,
   events.continent,
   events.channel,
   events.test,
   events.test_group,
   event_name,
   event_cnt as value
from events
union all
Select
   session.date,
   session.country,
   session.device,
   session.continent,
   session.channel,
   session.test,
   session.test_group,
   'session' as event_name,
   session_cnt as value
from session
union all
Select
   account.date,
   account.country,
   account.device,
   account.continent,
   account.channel,
   account.test,
   account.test_group,
   'new account' as event_name,
   new_account_cnt as value
from account
"""

query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()

##A/B Test Calculations

###Statistical Significance Analysis (Overall & by Test)

In [ ]:
events = ["add_payment_info", "add_shipping_info", "begin_checkout", "new account"]
tests = df["test"].unique()
results = []

for test in tests:
  for event in events:
    filtered_ev = df[(df["test"] == test) & (df["event_name"] == event)]
    filtered_s = df[(df["test"] == test) & (df["event_name"] == "session")]
    ev = filtered_ev["value"].sum()
    num = filtered_s["value"].sum()
    conv_rate = ev / num if num > 0 else 0

    filtered_ev_A = filtered_ev[filtered_ev["test_group"] == 1]
    filtered_s_A = filtered_s[filtered_s["test_group"] == 1]
    ev_A = filtered_ev_A["value"].sum()
    num_A = filtered_s_A["value"].sum()
    conv_rate_A = ev_A / num_A if num_A > 0 else 0

    filtered_ev_B = filtered_ev[filtered_ev["test_group"] == 2]
    filtered_s_B = filtered_s[filtered_s["test_group"] == 2]
    ev_B = filtered_ev_B["value"].sum()
    num_B = filtered_s_B["value"].sum()
    conv_rate_B = ev_B / num_B if num_B > 0 else 0

    metric_change = (conv_rate_B - conv_rate_A) / conv_rate_A * 100

    if num_A > 0 and num_B > 0:
      z_stat, p_value = sm.stats.proportions_ztest([ev_A, ev_B], [num_A, num_B])
    else:
      z_stat, p_value = np.nan, np.nan

    significant = "TRUE" if p_value < 0.05 else "FALSE"

    result = {
      "test": test,
      "metric": event,
      "total_num_of_event": ev,
      "total_num_of_session": num,
      "total_conversion": conv_rate,
      "A_total_num_of_event": ev_A,
      "A_num_of_session": num_A,
      "A_conversion": conv_rate_A,
      "B_num_of_event": ev_B,
      "B_num_of_session": num_B,
      "B_conversion": conv_rate_B,
      "metric_change": metric_change,
      "z_stat": z_stat,
      "p_value": p_value,
      "significant": significant
    }
    results.append(result)

df_results_total = pd.DataFrame(results)
df_results_total.head()

,test,metric,total_num_of_event,total_num_of_session,total_conversion,A_total_num_of_event,A_num_of_session,A_conversion,B_num_of_event,B_num_of_session,B_conversion,metric_change,z_stat,p_value,significant
0,2,add_payment_info,4753,100881,0.047115,2344,50637,0.046290,2409,50244,0.047946,3.576911,-1.240994,0.214608,FALSE
1,2,add_shipping_info,6990,100881,0.069290,3480,50637,0.068724,3510,50244,0.069859,1.650995,-0.709557,0.477979,FALSE
2,2,begin_checkout,8575,100881,0.085001,4262,50637,0.084168,4313,50244,0.085841,1.988164,-0.952898,0.340642,FALSE
3,2,new account,8349,100881,0.082761,4165,50637,0.082252,4184,50244,0.083274,1.241934,-0.588793,0.556000,FALSE
4,1,add_payment_info,4217,90555,0.046568,1988,45362,0.043825,2229,45193,0.049322,12.542021,-3.924884,0.000087,TRUE


In [ ]:
import os

path = "/content/drive/My Drive/Portfolio Project 2/AB Test Calculations (Total).xlsx"

directory = os.path.dirname(path)

os.makedirs(directory, exist_ok=True)

df_results_total.to_excel(path, index=False)

###Statistical Significance Analysis (Per Сontinent)

In [ ]:
events = ["add_payment_info", "add_shipping_info", "begin_checkout", "new account"]
tests = df["test"].unique()
continents = df["continent"].unique()

results = []

for continent in continents:
  for test in tests:
    for event in events:
      filtered_ev = df[(df["test"] == test) & (df["event_name"] == event)]
      filtered_s = df[(df["test"] == test) & (df["event_name"] == "session")]
      ev = filtered_ev["value"].sum()
      num = filtered_s["value"].sum()
      conv_rate = ev / num if num > 0 else 0

      filtered_ev_A = filtered_ev[filtered_ev["test_group"] == 1]
      filtered_s_A = filtered_s[filtered_s["test_group"] == 1]
      ev_A = filtered_ev_A["value"].sum()
      num_A = filtered_s_A["value"].sum()
      conv_rate_A = ev_A / num_A if num_A > 0 else 0

      filtered_ev_B = filtered_ev[filtered_ev["test_group"] == 2]
      filtered_s_B = filtered_s[filtered_s["test_group"] == 2]
      ev_B = filtered_ev_B["value"].sum()
      num_B = filtered_s_B["value"].sum()
      conv_rate_B = ev_B / num_B if num_B > 0 else 0

      metric_change = (conv_rate_B - conv_rate_A) / conv_rate_A * 100

      if num_A > 0 and num_B > 0:
        z_stat, p_value = sm.stats.proportions_ztest([ev_A, ev_B], [num_A, num_B])
      else:
        z_stat, p_value = np.nan, np.nan

      significant = "TRUE" if p_value < 0.05 else "FALSE"

      result = {
        "continent": continent,
        "test": test,
        "metric": event,
        "total_num_of_event": ev,
        "total_num_of_session": num,
        "total_conversion": conv_rate,
        "A_total_num_of_event": ev_A,
        "A_num_of_session": num_A,
        "A_conversion": conv_rate_A,
        "B_num_of_event": ev_B,
        "B_num_of_session": num_B,
        "B_conversion": conv_rate_B,
        "metric_change": metric_change,
        "z_stat": z_stat,
        "p_value": p_value,
        "significant": significant
      }
      results.append(result)

  df_results_per_continent = pd.DataFrame(results)
  df_results_per_continent.head()

In [ ]:
path = "/content/drive/My Drive/Portfolio Project 2/Statistical Significance Analysis (Per Сontinent).xlsx"

df_results_per_continent.to_excel(path, index=False)

###Statistical Significance Analysis (Per Device)

In [ ]:
events = ["add_payment_info", "add_shipping_info", "begin_checkout", "new account"]
tests = df["test"].unique()
devices = df["device"].unique()

results = []

for device in devices:
  for test in tests:
    for event in events:
      filtered_ev = df[(df["test"] == test) & (df["event_name"] == event)]
      filtered_s = df[(df["test"] == test) & (df["event_name"] == "session")]
      ev = filtered_ev["value"].sum()
      num = filtered_s["value"].sum()
      conv_rate = ev / num if num > 0 else 0

      filtered_ev_A = filtered_ev[filtered_ev["test_group"] == 1]
      filtered_s_A = filtered_s[filtered_s["test_group"] == 1]
      ev_A = filtered_ev_A["value"].sum()
      num_A = filtered_s_A["value"].sum()
      conv_rate_A = ev_A / num_A if num_A > 0 else 0

      filtered_ev_B = filtered_ev[filtered_ev["test_group"] == 2]
      filtered_s_B = filtered_s[filtered_s["test_group"] == 2]
      ev_B = filtered_ev_B["value"].sum()
      num_B = filtered_s_B["value"].sum()
      conv_rate_B = ev_B / num_B if num_B > 0 else 0

      metric_change = (conv_rate_B - conv_rate_A) / conv_rate_A * 100

      if num_A > 0 and num_B > 0:
        z_stat, p_value = sm.stats.proportions_ztest([ev_A, ev_B], [num_A, num_B])
      else:
        z_stat, p_value = np.nan, np.nan

      significant = "TRUE" if p_value < 0.05 else "FALSE"

      result = {
        "device": device,
        "test": test,
        "metric": event,
        "total_num_of_event": ev,
        "total_num_of_session": num,
        "total_conversion": conv_rate,
        "A_total_num_of_event": ev_A,
        "A_num_of_session": num_A,
        "A_conversion": conv_rate_A,
        "B_num_of_event": ev_B,
        "B_num_of_session": num_B,
        "B_conversion": conv_rate_B,
        "metric_change": metric_change,
        "z_stat": z_stat,
        "p_value": p_value,
        "significant": significant
      }
      results.append(result)

  df_results_per_device = pd.DataFrame(results)
  df_results_per_device.head()

In [ ]:
path = "/content/drive/My Drive/Portfolio Project 2/Statistical Significance Analysis (Per Device).xlsx"

df_results_per_device.to_excel(path, index=False)

###Statistical Significance Analysis (Per Test Channel)

In [ ]:
events = ["add_payment_info", "add_shipping_info", "begin_checkout", "new account"]
tests = df["test"].unique()
channels = df["channel"].unique()

results = []

for channel in channels:
  for test in tests:
    for event in events:
      filtered_ev = df[(df["test"] == test) & (df["event_name"] == event)]
      filtered_s = df[(df["test"] == test) & (df["event_name"] == "session")]
      ev = filtered_ev["value"].sum()
      num = filtered_s["value"].sum()
      conv_rate = ev / num if num > 0 else 0

      filtered_ev_A = filtered_ev[filtered_ev["test_group"] == 1]
      filtered_s_A = filtered_s[filtered_s["test_group"] == 1]
      ev_A = filtered_ev_A["value"].sum()
      num_A = filtered_s_A["value"].sum()
      conv_rate_A = ev_A / num_A if num_A > 0 else 0

      filtered_ev_B = filtered_ev[filtered_ev["test_group"] == 2]
      filtered_s_B = filtered_s[filtered_s["test_group"] == 2]
      ev_B = filtered_ev_B["value"].sum()
      num_B = filtered_s_B["value"].sum()
      conv_rate_B = ev_B / num_B if num_B > 0 else 0

      metric_change = (conv_rate_B - conv_rate_A) / conv_rate_A * 100

      if num_A > 0 and num_B > 0:
        z_stat, p_value = sm.stats.proportions_ztest([ev_A, ev_B], [num_A, num_B])
      else:
        z_stat, p_value = np.nan, np.nan

      significant = "TRUE" if p_value < 0.05 else "FALSE"

      result = {
        "channel": channel,
        "test": test,
        "metric": event,
        "total_num_of_event": ev,
        "total_num_of_session": num,
        "total_conversion": conv_rate,
        "A_total_num_of_event": ev_A,
        "A_num_of_session": num_A,
        "A_conversion": conv_rate_A,
        "B_num_of_event": ev_B,
        "B_num_of_session": num_B,
        "B_conversion": conv_rate_B,
        "metric_change": metric_change,
        "z_stat": z_stat,
        "p_value": p_value,
        "significant": significant
      }
      results.append(result)

  df_results_per_channel = pd.DataFrame(results)
  df_results_per_channel.head()

In [ ]:
path = "/content/drive/My Drive/Portfolio Project 2/Statistical Significance Analysis (Per Channel).xlsx"

df_results_per_channel.to_excel(path, index=False)

##A/B Test Calculation Results

Link to the CSV file with A/B test calculation results.

https://docs.google.com/spreadsheets/d/1a0e_T-OCwQgNOFKTorYXyymDmcO9_nH9/edit?usp=sharing&ouid=110399326020841856172&rtpof=true&sd=true

##Tableau Visualization

Dashboard for analyzing A/B test results with visualizations of key conversion metrics, enabling evaluation of variant performance and identification of statistically significant differences.

https://public.tableau.com/shared/JJRJD7ZCQ?:display_count=n&:origin=viz_share_link